In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pylab as plt
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
class myCallback (tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.95 and logs.get('val_accuracy')>0.95):
            print("\nReached 85% accuracy so cancelling training!")
            self.model.stop_training = True

In [8]:
def create_model():
    mobilenet_v3 = tf.keras.applications.MobileNetV3Small(input_shape=(224,224,3), include_top=False)

    model = tf.keras.Sequential([
        mobilenet_v3,
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(23,activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.summary()

    train_dir = '../data/training/'
    validation_dir = '../data/validation/'

    train_datagen = ImageDataGenerator(
        rescale=1. / 255.,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    validation_datagen = ImageDataGenerator(
        rescale=1. / 255.
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        batch_size=20,
        class_mode='categorical',
        target_size=(224, 224)
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        batch_size=20,
        class_mode='categorical',
        target_size=(224, 224)
    )

    callback = myCallback()

    model.fit(
        train_generator,
        epochs=300,
        validation_data=validation_generator,
        callbacks=callback
    )
    
    return model

In [10]:
model = create_model()
model.save("./modelv1-12.h5")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 batch_normalization_2 (Batc  (None, 7, 7, 576)        2304      
 hNormalization)                                                 
                                                                 
 global_average_pooling2d_2   (None, 576)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 576)               0         
                                                                 
 dense_2 (Dense)             (None, 23)                13271     
                                                      

In [22]:
import numpy as np
from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.models import load_model
import cv2

model = load_model("./modelv1-12.h5")

local_image_path = '../data/validation/Ayam Goreng/resep-ayam-goreng-kuning-tabur-serundeng-enak (1).jpg'

# Make predictions for the local image
# img = image.load_img(local_image_path, target_size=(224, 224))  # Adjust target_size as needed
img = cv2.imread(local_image_path)
img = cv2.resize(img, (224, 224))
img_array = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)
img_array = img_array/255.

# Make predictions
train_dir = "../data/training/"
classes = [f for f in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, f))]
predictions = model.predict(img_array)
print(predictions * 100)

predicted_class = np.argmax(predictions)
predicted_food = classes[predicted_class]

print(f"The predicted class is: {predicted_class}")
print(f"The predicted food is: {predicted_food}")

1/1 [==============================] - 1s 903ms/step
[[1.8974760e-04 5.9853565e-02 9.9482254e+01 1.6891776e-01 2.1800591e-04
  5.0852512e-04 2.2950245e-04 2.3596011e-04 5.3912308e-02 3.2319738e-05
  9.3239249e-04 7.9588633e-04 2.9688806e-04 1.4454729e-04 2.8299852e-04
  6.3364721e-05 1.9007125e-04 4.6953131e-04 3.4547187e-04 3.9201789e-02
  6.8232701e-03 2.5482537e-04 1.8383938e-01]]
The predicted class is: 2
The predicted food is: Ayam Goreng
